# Data processing

## Purpose
This notebook just process the raw data obtained from [SSP/SP](http://www.ssp.sp.gov.br/Estatistica/Pesquisa.aspx) into two datasets.

# Setup

## Library import
We import all the required Python libraries

In [24]:
import glob
import os
import warnings

from feature_engine.selection import DropConstantFeatures, DropDuplicateFeatures
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
import plotly.offline as ply
import cufflinks as cf

# Options for libraries
cf.go_offline(connected=True)
cf.set_config_file(theme='white')
pd.options.display.max_columns = None
pd.options.display.max_rows = None
plotly.offline.init_notebook_mode(connected=True)
plt.style.use('seaborn-paper')
warnings.filterwarnings("ignore")

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

## Custom functions

In [16]:
def create_df_from_data(file_name_pattern: str = 'Mensal-Capital*') -> pd.DataFrame:
    file_list = glob.glob(RAW_DATA_FOLDER + file_name_pattern + '.csv')
#     file_list.sort()

    df = pd.DataFrame()

    for file in file_list:
        year = file[-8:-4]
        df_temp = pd.read_csv(file, sep=';', na_values='...', thousands='.')
        df_temp.dropna(how='all', axis=1, inplace=True)
        df_temp.drop(columns=['Total'], inplace=True)
        df_temp.rename(columns={
            'Jan': year+'01', 'Fev': year+'02', 'Mar': year+'03', 'Abr': year+'04', 'Mai': year+'05', 'Jun': year+'06',
            'Jul': year+'07', 'Ago': year+'08', 'Set': year+'09', 'Out': year+'10', 'Nov': year+'11', 'Dez': year+'12'}, inplace=True)
        if df.empty:
            df = df_temp.copy()
        else:
            df = df.merge(df_temp, how='left', on='Ocorrencia')

    df = df.pivot_table(columns='Ocorrencia')
    return df

# Parameter definition
We set all relevant parameters for our notebook. By convention, parameters are uppercase, while all the 
other variables follow Python's guidelines.

In [17]:
# Data folders
RAW_DATA_FOLDER = "../data/raw/"
INTERIM_DATA_FOLDER = "../data/interim/"
PROCESSED_DATA_FOLDER = "../data/processed/"

RANDOM_STATE = 42


# Data import
We retrieve all the required data for the analysis.

In [18]:
# Mensal-Capital

df_crime = create_df_from_data('Mensal-Capital*')
        
print(f"""Qtd de meses e tipos de crimes: {df_crime.shape}""")
df_crime.sample(10, random_state=RANDOM_STATE)

Qtd de meses e tipos de crimes: (79, 23)


Ocorrencia,ESTUPRO,ESTUPRO DE VULNERÁVEL,FURTO - OUTROS,FURTO DE VEÍCULO,HOMICÍDIO CULPOSO OUTROS,HOMICÍDIO CULPOSO POR ACIDENTE DE TRÂNSITO,HOMICÍDIO DOLOSO (2),HOMICÍDIO DOLOSO POR ACIDENTE DE TRÂNSITO,LATROCÍNIO,LESÃO CORPORAL CULPOSA - OUTRAS,LESÃO CORPORAL CULPOSA POR ACIDENTE DE TRÂNSITO,LESÃO CORPORAL DOLOSA,LESÃO CORPORAL SEGUIDA DE MORTE,Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO (3),Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO POR ACIDENTE DE TRÂNSITO,Nº DE VÍTIMAS EM LATROCÍNIO,ROUBO - OUTROS,ROUBO A BANCO,ROUBO DE CARGA,ROUBO DE VEÍCULO,TENTATIVA DE HOMICÍDIO,TOTAL DE ESTUPRO (4),TOTAL DE ROUBO - OUTROS (1)
201707,65.0,132.0,15541.0,3538.0,1.0,37.0,44.0,2.0,10.0,64.0,1390.0,2265.0,0.0,49.0,2.0,10.0,11718.0,2.0,448.0,2619.0,51.0,197.0,12168.0
201501,NaN,NaN,13387.0,3818.0,2.0,26.0,97.0,0.0,9.0,85.0,1680.0,2675.0,NaN,102.0,0.0,9.0,NaN,7.0,514.0,3691.0,112.0,181.0,13188.0
201611,NaN,130.0,15482.0,3639.0,5.0,26.0,69.0,2.0,11.0,68.0,1281.0,2531.0,0.0,75.0,2.0,11.0,NaN,7.0,583.0,3205.0,74.0,205.0,13516.0
201708,74.0,116.0,16728.0,3751.0,2.0,22.0,51.0,2.0,7.0,125.0,1335.0,2500.0,0.0,58.0,2.0,7.0,12316.0,2.0,524.0,2597.0,55.0,190.0,12842.0
201607,NaN,0.0,15134.0,4038.0,1.0,26.0,72.0,1.0,9.0,74.0,1500.0,2274.0,0.0,74.0,1.0,9.0,NaN,2.0,473.0,3070.0,82.0,159.0,13016.0
201705,85.0,140.0,16486.0,3886.0,5.0,46.0,45.0,2.0,15.0,90.0,1440.0,2384.0,0.0,50.0,2.0,15.0,13372.0,5.0,493.0,2751.0,82.0,225.0,13870.0
201511,NaN,NaN,15006.0,3876.0,1.0,33.0,81.0,2.0,12.0,74.0,1490.0,2527.0,NaN,83.0,2.0,13.0,NaN,3.0,459.0,3437.0,81.0,174.0,13854.0
202011,48.0,162.0,13956.0,2455.0,4.0,40.0,51.0,0.0,4.0,82.0,983.0,2432.0,2.0,68.0,0.0,4.0,10771.0,1.0,273.0,1260.0,43.0,210.0,11045.0
201505,NaN,NaN,14858.0,3587.0,2.0,39.0,83.0,1.0,13.0,71.0,1886.0,2142.0,NaN,84.0,1.0,14.0,NaN,4.0,395.0,2843.0,83.0,164.0,12196.0
201601,NaN,0.0,13617.0,3393.0,3.0,21.0,58.0,1.0,9.0,72.0,1307.0,2241.0,0.0,59.0,1.0,9.0,NaN,3.0,458.0,3211.0,82.0,168.0,12645.0


In [19]:
# ProdutividadePolicial-Capital

df_prod_policial = create_df_from_data(file_name_pattern='ProdutividadePolicial-Capital*')

        
print(f"""Qtd de meses e tipos de ações policiais: {df_prod_policial.shape}""")
df_prod_policial.sample(10, random_state=RANDOM_STATE)

Qtd de meses e tipos de ações policiais: (79, 13)


Ocorrencia,Nº DE ARMAS DE FOGO APREENDIDAS,Nº DE FLAGRANTES LAVRADOS,Nº DE INFRATORES APREENDIDOS EM FLAGRANTE,Nº DE INFRATORES APREENDIDOS POR MANDADO,Nº DE PESSOAS PRESAS EM FLAGRANTE,Nº DE PESSOAS PRESAS POR MANDADO,Nº DE PRISÕES EFETUADAS,Nº DE VEÍCULOS RECUPERADOS,OCORRÊNCIAS DE APREENSÃO DE ENTORPECENTES(1),OCORRÊNCIAS DE PORTE DE ENTORPECENTES,OCORRÊNCIAS DE PORTE ILEGAL DE ARMA,OCORRÊNCIAS DE TRÁFICO DE ENTORPECENTES,TOT. DE INQUÉRITOS POLICIAIS INSTAURADOS
201707,297,2734,343,118,3262,1161,3176,2359,22,218,110,758,8488
201501,413,2223,245,27,2675,626,2528,3405,33,155,94,431,8323
201611,359,2761,348,51,3294,1101,2995,2570,23,263,106,657,8872
201708,283,2771,391,63,3266,1278,3071,2438,35,176,106,862,9253
201607,411,2952,326,25,3468,1117,3218,2721,16,442,146,721,9127
201705,338,3003,348,20,3362,1429,3402,2637,16,183,113,752,9936
201511,300,2469,384,36,2895,821,2799,3103,65,1399,113,534,8076
202011,219,1891,264,4,2205,571,2134,1155,19,98,73,589,6909
201505,353,2530,352,55,3084,835,2941,3060,61,416,106,660,8532
201601,313,2499,370,27,3014,822,2772,2911,13,948,117,644,7656


In [20]:
for item in ['ESTUPRO', 'ESTUPRO DE VULNERÁVEL', 'LESÃO CORPORAL SEGUIDA DE MORTE', 'ROUBO - OUTROS']:
    print(item, item in df_prod_policial.columns)

ESTUPRO False
ESTUPRO DE VULNERÁVEL False
LESÃO CORPORAL SEGUIDA DE MORTE False
ROUBO - OUTROS False


In [21]:
df_consolidado = pd.concat([df_prod_policial, df_crime], axis=1)
df_consolidado["ANO"] = df_consolidado.index.str[:4]
df_consolidado["MES"] = df_consolidado.index.str[-2:]

df_consolidado = df_consolidado[[
    'ANO', 'MES', 'Nº DE ARMAS DE FOGO APREENDIDAS', 'Nº DE FLAGRANTES LAVRADOS', 'Nº DE INFRATORES APREENDIDOS EM FLAGRANTE',
    'Nº DE INFRATORES APREENDIDOS POR MANDADO', 'Nº DE PESSOAS PRESAS EM FLAGRANTE', 'Nº DE PESSOAS PRESAS POR MANDADO',
    'Nº DE PRISÕES EFETUADAS', 'Nº DE VEÍCULOS RECUPERADOS', 'OCORRÊNCIAS DE APREENSÃO DE ENTORPECENTES(1)',
    'OCORRÊNCIAS DE PORTE DE ENTORPECENTES', 'OCORRÊNCIAS DE PORTE ILEGAL DE ARMA', 'OCORRÊNCIAS DE TRÁFICO DE ENTORPECENTES',
    'TOT. DE INQUÉRITOS POLICIAIS INSTAURADOS', 'ESTUPRO', 'ESTUPRO DE VULNERÁVEL', 'FURTO - OUTROS', 'FURTO DE VEÍCULO',
    'HOMICÍDIO CULPOSO OUTROS', 'HOMICÍDIO CULPOSO POR ACIDENTE DE TRÂNSITO', 'HOMICÍDIO DOLOSO (2)',
    'HOMICÍDIO DOLOSO POR ACIDENTE DE TRÂNSITO', 'LATROCÍNIO', 'LESÃO CORPORAL CULPOSA - OUTRAS','LESÃO CORPORAL CULPOSA POR ACIDENTE DE TRÂNSITO',
    'LESÃO CORPORAL DOLOSA', 'LESÃO CORPORAL SEGUIDA DE MORTE', 'Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO (3)',
    'Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO POR ACIDENTE DE TRÂNSITO', 'Nº DE VÍTIMAS EM LATROCÍNIO', 'ROUBO - OUTROS', 'ROUBO A BANCO',
    'ROUBO DE CARGA', 'ROUBO DE VEÍCULO', 'TENTATIVA DE HOMICÍDIO', 'TOTAL DE ESTUPRO (4)', 'TOTAL DE ROUBO - OUTROS (1)']].copy()

print(f"""Dimensões da df resultante: {df_consolidado.shape}""")
df_consolidado.head()

Dimensões da df resultante: (79, 38)


Ocorrencia,ANO,MES,Nº DE ARMAS DE FOGO APREENDIDAS,Nº DE FLAGRANTES LAVRADOS,Nº DE INFRATORES APREENDIDOS EM FLAGRANTE,Nº DE INFRATORES APREENDIDOS POR MANDADO,Nº DE PESSOAS PRESAS EM FLAGRANTE,Nº DE PESSOAS PRESAS POR MANDADO,Nº DE PRISÕES EFETUADAS,Nº DE VEÍCULOS RECUPERADOS,OCORRÊNCIAS DE APREENSÃO DE ENTORPECENTES(1),OCORRÊNCIAS DE PORTE DE ENTORPECENTES,OCORRÊNCIAS DE PORTE ILEGAL DE ARMA,OCORRÊNCIAS DE TRÁFICO DE ENTORPECENTES,TOT. DE INQUÉRITOS POLICIAIS INSTAURADOS,ESTUPRO,ESTUPRO DE VULNERÁVEL,FURTO - OUTROS,FURTO DE VEÍCULO,HOMICÍDIO CULPOSO OUTROS,HOMICÍDIO CULPOSO POR ACIDENTE DE TRÂNSITO,HOMICÍDIO DOLOSO (2),HOMICÍDIO DOLOSO POR ACIDENTE DE TRÂNSITO,LATROCÍNIO,LESÃO CORPORAL CULPOSA - OUTRAS,LESÃO CORPORAL CULPOSA POR ACIDENTE DE TRÂNSITO,LESÃO CORPORAL DOLOSA,LESÃO CORPORAL SEGUIDA DE MORTE,Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO (3),Nº DE VÍTIMAS EM HOMICÍDIO DOLOSO POR ACIDENTE DE TRÂNSITO,Nº DE VÍTIMAS EM LATROCÍNIO,ROUBO - OUTROS,ROUBO A BANCO,ROUBO DE CARGA,ROUBO DE VEÍCULO,TENTATIVA DE HOMICÍDIO,TOTAL DE ESTUPRO (4),TOTAL DE ROUBO - OUTROS (1)
201501,2015,01,413,2223,245,27,2675,626,2528,3405,33,155,94,431,8323,NaN,NaN,13387.0,3818.0,2.0,26.0,97.0,0.0,9.0,85.0,1680.0,2675.0,NaN,102.0,0.0,9.0,NaN,7.0,514.0,3691.0,112.0,181.0,13188.0
201502,2015,02,363,2157,269,34,2591,586,2428,3188,51,241,109,506,8017,NaN,NaN,12931.0,3611.0,3.0,30.0,93.0,1.0,8.0,68.0,1574.0,2396.0,NaN,98.0,1.0,8.0,NaN,10.0,431.0,3261.0,70.0,163.0,12393.0
201503,2015,03,469,2605,319,54,3062,818,3026,3352,73,291,148,661,9523,NaN,NaN,16226.0,4253.0,4.0,49.0,103.0,0.0,12.0,83.0,2004.0,2592.0,NaN,114.0,0.0,12.0,NaN,7.0,533.0,3376.0,79.0,197.0,14217.0
201504,2015,04,392,2408,327,30,2895,871,2767,2952,68,335,125,627,7835,NaN,NaN,15233.0,3805.0,3.0,31.0,81.0,0.0,9.0,88.0,1756.0,2351.0,NaN,97.0,0.0,9.0,NaN,12.0,463.0,3214.0,86.0,167.0,12947.0
201505,2015,05,353,2530,352,55,3084,835,2941,3060,61,416,106,660,8532,NaN,NaN,14858.0,3587.0,2.0,39.0,83.0,1.0,13.0,71.0,1886.0,2142.0,NaN,84.0,1.0,14.0,NaN,4.0,395.0,2843.0,83.0,164.0,12196.0


# Data export

Creating and exporting interim datasets for data viz.

In [26]:
df_crime.to_excel(INTERIM_DATA_FOLDER+'df_crime.xlsx')
df_prod_policial.to_excel(INTERIM_DATA_FOLDER+'df_prod_policial.xlsx')

df_consolidado.to_parquet(INTERIM_DATA_FOLDER+'df_consolidado.pqt')